In [ ]:
# ! pip install pandas
# ! pip install 'thefuzz[speedup]'

In [ ]:
import pandas as pd
from thefuzz import fuzz
from thefuzz import process

In [ ]:
df_price = pd.read_csv('seeds/fruit_prices_fact.csv')
df_price

In [ ]:
df_input = pd.read_csv('seeds/fruit_user_input.csv')
df_input

In [ ]:
def custom_scorer(string):
    '''
    for a given string
    return the best match out of the `fruit_name` column in the df_to table
    '''
    
    x = process.extractOne(string,df_price["fruit_name"], score_cutoff=60)
    
    if x is not None:
        return x[0]
    else:
        return None

In [ ]:
df_final = (df_input
           # make new col, `fruit_name`, with best match against actual table
           .assign(fruit_name = lambda df: df['fruit_user_input'].apply(custom_scorer))
           # join the actual fruit price table
           .merge(df_price, on="fruit_name")
           # # calculate subtotal
           .assign(total= lambda df: df.quantity * df.cost)
           # # find total for each user and sort descending by total price
           .groupby("user_name")['total'].sum()
           .reset_index()
           .sort_values("total", ascending=False)
          )
df_final